In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans


station_geo = pd.read_csv('station_geo.csv')
station_geo_numpy = station_geo.to_numpy()
print('station_geo \n', station_geo)
zones = station_geo.district_n.unique()
zones.sort()
print('Zones : \n ',zones)
zoneNumbers = len(zones)
print('Number of stations : ', len(zones))

od_matrix = np.zeros((zoneNumbers, zoneNumbers))
od_matrix_df = pd.DataFrame(od_matrix, index=zones, columns=zones)
print('od_matrix_df \n', od_matrix_df)

bus_records_raw = pd.read_csv('bus7.csv')
print ('bus_records_raw \n',bus_records_raw)
bus_records = bus_records_raw.drop_duplicates()
print ('bus_records after remove duplicates \n',bus_records)
bus_records = bus_records.sort_values(by=['FDate','LineCode','plat','Time'], inplace=False,
               ascending = [True, True,True,True])

print("Get the bus_records after sorting:\n", bus_records)

routes = pd.read_csv('routes.csv')
print("Bus routes \n",routes)

metro_records_raw = pd.read_csv('metro7.csv')
print ('metro_records_raw \n',metro_records_raw)
metro_records = metro_records_raw.drop_duplicates()
print ('metro_records after remove duplicates \n',metro_records)
metro_records = metro_records.sort_values(by=['FDate','gate_id','Time'], inplace=False,
               ascending = [True, True,True])

print("Get the metro_records after sorting:\n", metro_records)


unique_Bus_Fdates = bus_records.FDate.unique()
print("Fdates in Bus records \n",unique_Bus_Fdates)


unique_metro_Fdates = metro_records.FDate.unique()
print("Fdates in metro records \n",unique_metro_Fdates)

for h in range(len(unique_Bus_Fdates)):
    print('**** FDate**** ',unique_Bus_Fdates[h])
    print('Bus transactions in fdate',unique_Bus_Fdates[h],'\n')  
    bus_transactions = bus_records.loc[bus_records['FDate'] == unique_Bus_Fdates[h]]
    print(bus_transactions)
    
    
    print('Metro transactions in fdate',unique_Bus_Fdates[h],'\n')  
    metro_transactions = metro_records.loc[metro_records['FDate'] == unique_Bus_Fdates[h]]
    print(metro_transactions)
    
    #list of active bus lines in selected Fdate
    l = bus_transactions.LineCode.unique()
    print('unique Line Codes: \n ',sorted(l))

    print('unique plats: \n ',bus_transactions.plat.unique())

    buses_with_station = pd.DataFrame(columns=["CardID","LineCode","FDate","Time","plat","Station"])

    for i in range(len(l)):
       # print('transactions in lines',l[i])
        s = bus_transactions.loc[bus_transactions['LineCode'] == l[i]]


        line_num =str(int(l[i]))
        print('stations in a specefic bus line',line_num)
        filter_go_stations = routes.loc[(routes['line'] == line_num) & (routes['go_back'] =='go')]
        filter_back_stations = routes.loc[(routes['line'] == line_num) & (routes['go_back'] =='back')]
        filter_go_back_stations = routes.loc[(routes['line'] == line_num) & (routes['go_back'] =='go_back')]
        print ('len-of-go stations:',len(filter_go_stations))
        print ('len-of-back stations:',len(filter_back_stations))
        print ('len-of-go back stations:',len(filter_go_back_stations))

        if(len(filter_go_stations)==0 or len(filter_back_stations)==0):
            continue

        orig_station = filter_go_stations.to_numpy()[0][0]
        dest_station =  filter_back_stations.to_numpy()[0][0]
        print('first Station ',filter_go_stations.to_numpy()[0],filter_go_stations.to_numpy()[0][0])
        print('last Station ',filter_back_stations.to_numpy()[0])

        print(s.plat.unique())
        p = s.plat.unique()

        for j in range(len(p)):

            print('transactions in line',l[i],'plat',p[j])
            ps  = bus_transactions.loc[(bus_transactions['LineCode'] == l[i]) & (bus_transactions['plat'] ==p[j])]

            print('ps: \n ',ps)

            ps.to_csv('ps_test.csv')

            # For each buckets of transactions in a specific line and plat, we try to  cluster transaction in related station
            # ps evaluate to buses in my previous code

            buses_numpy = ps.to_numpy()
            print('first row of the selected chunck : \n',buses_numpy[0])

            t0= buses_numpy[0][3]
            ts=buses_numpy[len(buses_numpy)-1][3]
            print(len(buses_numpy))
            print('t0 =',t0)
            print('ts =',ts)
            time_interval = 600 #second for finding orgin or destination stations(over 10 minute)
            interval_counts=round((ts-t0)/time_interval)
            print('time_intervals',round((ts-t0)/time_interval))
            counter=1
            buses_counter=1
            sum_count=0
            interval_numpy = np.ones((interval_counts,2))

            for k in range(len(interval_numpy)):
                while buses_numpy[buses_counter][3]<(t0):
                    sum_count= sum_count+1
                    #print(buses_numpy[buses_counter][3],t0)
                    #print('sum_count=',sum_count)
                    buses_counter = buses_counter+1
                interval_numpy[k]=(t0,sum_count)
                t0 = t0+time_interval
                sum_count =0
            #print('10min interval view \n for line',l[i],'plat',p[j],' \n',interval_numpy)
            np.savetxt("interval_numpy.csv", interval_numpy, delimiter=",")
            interval_numpy2 = interval_numpy.copy()

            #Calculate 10min intervals to find origin or destinations - Each row means a go or back
            df_bus_service = pd.DataFrame(columns=["time_in","time_out"])


            len_interval_numpy =  len(interval_numpy) 

            for k in range(len(interval_numpy2)):
                #print ('k ',k)
                if interval_numpy[k][1]==0: 
                    if k==0: #first row
                        in_time = interval_numpy2[k][0]
                        out_time =interval_numpy2[k][0]+ time_interval
                        bus_services_row =[in_time,out_time]
                        df_bus_service.loc[len(df_bus_service)] = bus_services_row   
                        #print('in if \n',df_bus_service)
                    elif interval_numpy2[k-1][1]==0:
                        in_time = interval_numpy2[k-1][0]
                        out_time =interval_numpy2[k][0]+ time_interval
                        bus_services_row =[in_time,out_time]
                        df_bus_service.loc[len(df_bus_service)] = bus_services_row  
                        df_bus_service.loc[df_bus_service['time_in'] == in_time] = bus_services_row
                        interval_numpy2[k][0] = interval_numpy2[k-1][0]  
                        #print('finded in data : ',df_bus_service2.loc[df_bus_service2['time_in'] == in_time])
                        #print('in elif else \n',df_bus_service)
                    else:
                        in_time = interval_numpy2[k][0]
                        out_time =interval_numpy2[k][0]+ time_interval
                        bus_services_row =[in_time,out_time]
                        df_bus_service.loc[len(df_bus_service)] = bus_services_row

                        #print('in else \n',df_bus_service)
            df_bus_service = df_bus_service.drop_duplicates()
            #print('Origin destination interval view \n',df_bus_service)
            df_bus_service = df_bus_service.reset_index(drop=True)
            print('Origin destination interval view reindex \n',df_bus_service)
            df_bus_service.to_csv('origin_destiniation_head_times_test.csv')                                  

            #Assign origin and destination stations to the transactions
          #  buses_with_station = pd.DataFrame(columns=["TID","Time","StationID","Is_OD"])
            station_Counter = 1
            station_org_or_des = orig_station
            #buses_with_station = pd.DataFrame(columns=["CardID","LineCode","FDate","Time","plat","Station"])

            for k in range(len(df_bus_service)):     #Orgin_destination Staions
                time_in_od = df_bus_service.loc[k][0] #start_in orgin or destination
                time_out_od = df_bus_service.loc[k][1]#end_out orgin or destination
                #print('time_in_od,time_out_od:',time_in_od,time_out_od)

                if station_Counter<0:
                    station_org_or_des=dest_station
                else:
                    station_org_or_des = orig_station

                for m in range(len(buses_numpy)):
                    #print(buses_numpy[j][1])
                    #buses_numpy[i][1]<=(time_in_od) and [bus_counter][1]>(time_out_od)
                    #print('added',buses_numpy[bus_counter][1])
                    #bus_station_row = [buses_numpy[bus_counter][0],buses_numpy[bus_counter][1],station_Counter,'yes']

                    if(buses_numpy[m][3]>=(time_in_od) and buses_numpy[m][3]<=(time_out_od)):
                        #print([buses_numpy[j][0],buses_numpy[j][1],station_Counter,'yes'])
                        buses_with_station.loc[len(buses_with_station)]=[buses_numpy[m][0],buses_numpy[m][1],buses_numpy[m][2],buses_numpy[m][3],buses_numpy[m][4],station_org_or_des]
                        #print('numpy len before delete:',len(buses_numpy))

                        #print('np delete',np.delete(buses_numpy, m, axis=0))
                        #print('numpy len after delete:',len(buses_numpy))
                station_Counter= -station_Counter
                #print('station_counter',station_Counter)
            #first attempt to update stations of transactions - only orgin or destination
            print('OD bus stations of transactions: \n',buses_with_station)
            buses_with_station.to_csv('OD_BUS_Stations_test.csv')

            station_Counter = 1

            ##dd
            for k in range(len(df_bus_service)-1):     #Other Staions

                time_start = df_bus_service.loc[k][1]#end_out orgin or destination
                time_stop = df_bus_service.loc[k+1][0] #start_in orgin or destination
                #print('time_start','time_stop',time_start,time_stop)
                station_Counter = station_Counter+1
                df_bus_partial = pd.DataFrame(columns=["CardID","LineCode","FDate","Time","plat"])
                for m in range(len(buses_numpy)):
                    #print(buses_numpy[j][1])
                    #buses_numpy[i][1]<=(time_in_od) and [bus_counter][1]>(time_out_od)
                    #print('added',buses_numpy[bus_counter][1])
                    #bus_station_row = [buses_numpy[bus_counter][0],buses_numpy[bus_counter][1],station_Counter,'yes']

                    if(buses_numpy[m][3]>(time_start) and buses_numpy[m][3]<(time_stop)):
                        #print([buses_numpy[j][0],buses_numpy[j][1],station_Counter,'no'])
                        df_bus_partial.loc[len(df_bus_partial)]=[buses_numpy[m][0],buses_numpy[m][1],buses_numpy[m][2],buses_numpy[m][3],buses_numpy[m][4]]
                       # buses_with_station.loc[len(buses_with_station)]=[buses_numpy[j][0],buses_numpy[j][1],station_Counter,'yes']
                #print('df_bus_partial : \n',df_bus_partial)

                partial_times = df_bus_partial[["Time","LineCode"]]
                numpy_df_bus_partial = df_bus_partial.to_numpy()
                #print(partial_times)


                if station_Counter<0: #Back
                  station_org_or_des=dest_station

                  if(len(df_bus_partial) >=len(filter_back_stations)):
                    #print('if(len(df_bus_partial) >=len(filter_back_stations))')
                    #We remove OD destinatio form filter_back_stations 
                    kmeans = KMeans(n_clusters=len(filter_back_stations)-1,random_state=0).fit(partial_times.to_numpy())
                    #print('kmeans.labels_ in back routes',kmeans.labels_)

                    kmeansLabels = pd.DataFrame(kmeans.labels_,columns=["label"])
                    print('kmeansLabels \n',kmeansLabels.label.unique())
                    uniqLabels = kmeansLabels.label.unique()
                    for n in range(len(uniqLabels)):
                      #print(uniqLabels[n])
                      filteredLabels = kmeansLabels.loc[kmeansLabels['label'] == uniqLabels[n]]
                      #print('filteredLabels',filteredLabels)
                      for z in range(len(filteredLabels)):
                        buses_with_station.loc[len(buses_with_station)]=[numpy_df_bus_partial[z][0],numpy_df_bus_partial[z][1],numpy_df_bus_partial[z][2],numpy_df_bus_partial[z][3],numpy_df_bus_partial[z][4],filter_go_stations.to_numpy()[n+1][0]]
                        #print('partial entery :',numpy_df_bus_partial[z][0],numpy_df_bus_partial[z][1],numpy_df_bus_partial[z][2],numpy_df_bus_partial[z][3],numpy_df_bus_partial[z][4],filter_go_stations.to_numpy()[n+1][0])

                else:   #Go
                  station_org_or_des = orig_station
                  if(len(df_bus_partial) >=len(filter_go_stations)):
                    #print('if(len(df_bus_partial) >=len(filter_go_stations))')
                    #We remove OD destinatio form filter_back_stations
                    kmeans = KMeans(n_clusters=len(filter_go_stations)-1,random_state=0).fit(partial_times.to_numpy())
                    #print('kmeans.labels_ in go routes',kmeans.labels_)

                    kmeansLabels = pd.DataFrame(kmeans.labels_,columns=["label"])
                    #print('kmeansLabels \n',kmeansLabels.label.unique())
                    uniqLabels = kmeansLabels.label.unique()
                    for n in range(len(uniqLabels)):
                      #print(uniqLabels[n])
                      filteredLabels = kmeansLabels.loc[kmeansLabels['label'] == uniqLabels[n]]
                      #print('filteredLabels',filteredLabels)
                      for z in range(len(filteredLabels)):
                        buses_with_station.loc[len(buses_with_station)]=[numpy_df_bus_partial[z][0],numpy_df_bus_partial[z][1],numpy_df_bus_partial[z][2],numpy_df_bus_partial[z][3],numpy_df_bus_partial[z][4],filter_go_stations.to_numpy()[n+1][0]]
                        #print('partial entery :',numpy_df_bus_partial[z][0],numpy_df_bus_partial[z][1],numpy_df_bus_partial[z][2],numpy_df_bus_partial[z][3],numpy_df_bus_partial[z][4],filter_go_stations.to_numpy()[n+1][0])
                      #print('bus partial with all stations : \n',df_bus_partial)
                #df_bus_partial.to_csv("bus_partial"+str(i)+".csv")

                #kmeans = KMeans(n_clusters=10).fit(df_bus_partial.to_numpy())


                #print(kmeans.labels_)
        #buses_with_station.to_csv("bus_with_station980623_L"+str(l[i])+".csv")
    #buses_with_station.to_csv("bus_with_station_"+ str(l[i])+"_"+ str(p[j])+".csv")
    buses_with_station.to_csv("bus_with_station"+str(h)+".csv")
    
    #Now we try to integrate bus transactions  with merto transactions on the stations
    
    buses_with_station = buses_with_station.drop_duplicates()
    #print ('bus_with_station \n ', bus_with_station)

    bus_with_station_filter = buses_with_station[["CardID", "FDate","Time","Station"]]
    print('bus_with_station_filter \n',bus_with_station_filter)

    #metro_transactions = metro_transactions.rename(columns = {'gate_id':'Station'}, inplace = True)
    metro_transactions = metro_transactions.reindex(columns=['CardID', 'FDate', 'Time','gate_id'])
    metro_transactions.columns = metro_transactions.columns.str.replace("gate_id", "Station")
    print('metro_transactions \n ', metro_transactions)

    metro_bus_merge = pd.concat([metro_transactions, bus_with_station_filter])
    print('concat \n', metro_bus_merge)

    uniqueCardIds = metro_bus_merge.CardID.unique()

    print('unique cardIDs: \n ',metro_bus_merge.CardID.unique())
    print(len(uniqueCardIds))


    for i in range(len(uniqueCardIds)):
       # print('transactions in lines',l[i])
        s = metro_bus_merge.loc[metro_bus_merge['CardID'] == uniqueCardIds[i]]
        s = s.sort_values('Time')
        s_numpy = s.to_numpy()
        if(len(s)>1 and len(s)<9):
            print(' \n CardID : ',uniqueCardIds[i])
            print('With stations: \n',s)
            #print('Without staions: \n',bus_transactions.loc[bus_transactions['CardID'] == uniqueCardIds[i]])

            for j in range(len(s)-1):
                if ((s_numpy[j+1][2]- s_numpy[j][2])>600):#10min
                    print('we find an arch : ', s_numpy[j][3],'-->', s_numpy[j+1][3])
                    #maxtrix_OD_desc.loc[len(maxtrix_OD_desc)]=[s_numpy[j][1],s_numpy[j][3],s_numpy[j][2],s_numpy[j][5],s_numpy[j][4],s_numpy[j][6],s_numpy[j+1][2],s_numpy[j+1][5],s_numpy[j+1][4],s_numpy[j+1][6]]
                    #od_matrix[1096][1096] = 3
                    s1  = station_geo.loc[(station_geo['station_co'] == s_numpy[j][3])]
                    s2=   station_geo.loc[(station_geo['station_co'] == s_numpy[j+1][3])]
                    print('s1 = \n ',s1)
                    print('s2 = \n',s2)
                    z1 = int(s1.to_numpy()[0][1])
                    z2 = int(s2.to_numpy()[0][1])
                    if z1 != z2:
                       print('z1,z2 ',z1,':',z2)
                       print(od_matrix_df.at[z1, z2])
                       od_matrix_df.at[z1, z2] = od_matrix_df.at[z1, z2]+1
    print('mod_matrix: after  \n ',od_matrix_df)
    od_matrix_df.to_csv("od_matrix_df_"+str(h)+".csv")    
od_matrix_df.to_csv("od_matrix_df.csv")

station_geo 
       station_co  district_n          x           y
0           1129           7  563030.95  3615891.13
1           1154           7  564271.98  3615841.64
2           1155           7  564024.45  3615856.68
3           1190           7  563032.43  3615642.03
4           1212           7  563608.63  3615871.86
...          ...         ...        ...         ...
2725         132           8  560349.63  3619997.18
2726         133           8  561637.98  3619868.29
2727         134           8  561637.98  3619868.29
2728         135           8  562395.97  3618967.34
2729         137           8  562843.09  3618183.53

[2730 rows x 4 columns]
Zones : 
  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Number of stations :  16
od_matrix_df 
      1    2    3    4    5    6    7    8    9    10   11   12   13   14   15  \
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

             CardID  LineCode       FDate   Time  plat
328847    542446139         1  1398/06/23  24030  A145
328848   2620656172         1  1398/06/23  24302  A145
328849   3446252780         1  1398/06/23  26200  A145
328850   1281272526         1  1398/06/23  26446  A145
328851    981010891         1  1398/06/23  26449  A145
...             ...       ...         ...    ...   ...
3347256  3256171325      1019  1398/06/23  26450   A23
3347257   636295002      1019  1398/06/23  26572   A23
3347258  3104840174      1019  1398/06/23  26961   A23
3347259   636322666      1019  1398/06/23  26966   A23
3347260  3256171325      1019  1398/06/23  26999   A23

[470576 rows x 5 columns]
Metro transactions in fdate 1398/06/23 

            CardID  gate_id       FDate   Time
115624  3553272220      126  1398/06/23  22910
117207  3151596446      126  1398/06/23  22917
117208  3794635356      126  1398/06/23  22925
117209   293791323      126  1398/06/23  22930
115354  2681130596      126  1398/06/

OD bus stations of transactions: 
         CardID LineCode       FDate   Time  plat Station
0    542446139        1  1398/06/23  24030  A145    3169
1   2620656172        1  1398/06/23  24302  A145    3169
2   3446252780        1  1398/06/23  26200  A145    1227
3      7900260        1  1398/06/23  29677  A145    3169
4    361857972        1  1398/06/23  29720  A145    3169
..         ...      ...         ...    ...   ...     ...
61  3446907884        1  1398/06/23  74398  A145    1227
62  4268576994        1  1398/06/23  74400  A145    1227
63   721882830        1  1398/06/23  74402  A145    1227
64   721882830        1  1398/06/23  74403  A145    1227
65   175996118        1  1398/06/23  74407  A145    1227

[66 rows x 6 columns]


KeyboardInterrupt: 